In [3]:
#测试1：l0_time
import tensorflow as tf
batch_size=1
num_point=2
nsampoint=1
num_frames=5
print("一个样本中，原始点云序列点总数ndataset={}*{}".format(num_frames,num_point))
print("经过FPS采样后，一个样本中点总数npoint={}*{}".format(num_frames,nsampoint))
l0_time = tf.concat([tf.ones([batch_size, num_point, 1]) * i for i in range(num_frames)], \
            axis=-2) #shape(batch_size, ndataset, 1)  (1,3*2,1)  num_frames*num_point=ndataset
new_time=tf.concat([tf.ones([batch_size, nsampoint, 1]) * i for i in range(num_frames)], \
            axis=-2)  #shape(batch_size, npoint, 1)  (1,3*1,1)   num_frames*nsampoint= npoint
l0_time_reshape=tf.reshape(l0_time, [batch_size, 1, -1])  #(1,1,3*2)
new_time_=tf.abs(l0_time_reshape-new_time)  #时间差：(1,3*1,3*2) (batch_size,npoint,dataset)
cast_time=tf.cast(new_time_, tf.int32)  #new_time_值取整变为tf.int32类型
with tf.Session() as sess:
    #b=sess.run(l0_time)
    #print(sess.run(l0_time))
    #print(sess.run(l0_time_reshape))
    print("new_time_时间差:\n",sess.run(new_time_))
    print("new_time_时间差取整:\n",sess.run(cast_time))





一个样本中，原始点云序列点总数ndataset=5*2
经过FPS采样后，一个样本中点总数npoint=5*1
new_time_时间差:
 [[[0. 0. 1. 1. 2. 2. 3. 3. 4. 4.]
  [1. 1. 0. 0. 1. 1. 2. 2. 3. 3.]
  [2. 2. 1. 1. 0. 0. 1. 1. 2. 2.]
  [3. 3. 2. 2. 1. 1. 0. 0. 1. 1.]
  [4. 4. 3. 3. 2. 2. 1. 1. 0. 0.]]]
new_time_时间差取整:
 [[[0 0 1 1 2 2 3 3 4 4]
  [1 1 0 0 1 1 2 2 3 3]
  [2 2 1 1 0 0 1 1 2 2]
  [3 3 2 2 1 1 0 0 1 1]
  [4 4 3 3 2 2 1 1 0 0]]]


In [52]:
# #测试1：tf.ones([batch_size, num_point, 1]),及其*i
# import tensorflow as tf
# batch_size=1
# num_point=2
# num_frames=3
# a=tf.ones([batch_size, num_point, 1])
#
# with tf.Session() as sess:
#     print('a=tf.ones([batch_size, num_point, 1]):\n',sess.run(a))
#     for i in range(num_frames):
#         print('a*{}=\n{}'.format(i,sess.run(a*i)))   #a*i

a=tf.ones([batch_size, num_point, 1]):
 [[[1.]
  [1.]]]
a*0=
[[[0.]
  [0.]]]
a*1=
[[[1.]
  [1.]]]
a*2=
[[[2.]
  [2.]]]


In [4]:
    #测试2:RADIUS1/2
    import numpy as np
    num_frames=5
    print("时间，一个样本帧数：",num_frames)
    RADIUS1 = np.linspace(0.5, 0.6, num_frames, dtype='float32')#0.5开始，0.6结束，共num_frames多个数
    #RADIUS2 = RADIUS1 * 2
    print('RADIUS1:',RADIUS1)
    #print('RADIUS2:',RADIUS2)
    #测试radius[delta_t:]
    delta_t=2
    radius=RADIUS1
    radius[delta_t:] = 0
    print('RADIUS1[2:]=0之后:',RADIUS1)
    # radius=RADIUS2
    # radius[delta_t:] = 0
    # print('RADIUS2[2:]=0之后:',RADIUS2)

时间，一个样本帧数： 5
RADIUS1: [0.5   0.525 0.55  0.575 0.6  ]
RADIUS1[2:]=0之后: [0.5   0.525 0.    0.    0.   ]


In [5]:
#测试3：radius_=tf.gather(radius,
print("令batch_size=1,即以一个样本，一个动态手势视频为例")
print("一个样本中，原始点云序列点总数ndataset={}*{}".format(num_frames,num_point))#来自测试1
print("经过FPS采样后，一个样本中点总数npoint={}*{}".format(num_frames,nsampoint))#来自测试1
print("时间，一个样本帧数：",num_frames)  #来自测试2
print('radius:\n',radius)   #来自测试2：shape:num_frame ,32.本测试中是3
cast_time=tf.cast(new_time_, tf.int32)   #时间差：来自测试1:shape:(batch_size,npoints,ndataset),其中npoints:32*32,ndataset=32*128. 测试中（1,3*1,3*2）
sub_time=cast_time
radius_=tf.gather(radius, sub_time)

with tf.Session() as sess:
    print("sub_time:\n",sess.run(sub_time))
    print("radius_:\n",sess.run(radius_))






令batch_size=1,即以一个样本，一个动态手势视频为例
一个样本中，原始点云序列点总数ndataset=5*2
经过FPS采样后，一个样本中点总数npoint=5*1
时间，一个样本帧数： 5
radius:
 [0.5   0.525 0.    0.    0.   ]
sub_time:
 [[[0 0 1 1 2 2 3 3 4 4]
  [1 1 0 0 1 1 2 2 3 3]
  [2 2 1 1 0 0 1 1 2 2]
  [3 3 2 2 1 1 0 0 1 1]
  [4 4 3 3 2 2 1 1 0 0]]]
radius_:
 [[[0.5   0.5   0.525 0.525 0.    0.    0.    0.    0.    0.   ]
  [0.525 0.525 0.5   0.5   0.525 0.525 0.    0.    0.    0.   ]
  [0.    0.    0.525 0.525 0.5   0.5   0.525 0.525 0.    0.   ]
  [0.    0.    0.    0.    0.525 0.525 0.5   0.5   0.525 0.525]
  [0.    0.    0.    0.    0.    0.    0.525 0.525 0.5   0.5  ]]]


In [8]:
#测试4:tf.expand_dims，tf.tile
#new_xyz(batch_size, npoint, 3)  比如：npoint（32*32）就是32*32个采样点的3维坐标
batch_size=1
npoint=2
nsample=5

new_xyz=tf.zeros([batch_size,npoint,3])  #3
new_xyz_expand=tf.expand_dims(new_xyz, 2)#3->(1,3)
new_xyz_tile=tf.tile(tf.expand_dims(new_xyz, 2), [1, 1, nsample, 1]) #(1,3)->(nsample,3)

with tf.Session() as sess:
    ori=sess.run(new_xyz)
    exp=sess.run(new_xyz_expand)
    tile=sess.run(new_xyz_tile)
    print("new_xyz的shape:",ori.shape)
    print("new_xyz_expand的shape:",exp.shape)
    print("new_xyz_tile的shape:",tile.shape)



new_xyz的shape: (1, 2, 3)
new_xyz_expand的shape: (1, 2, 1, 3)
new_xyz_tile的shape: (1, 2, 5, 3)


In [18]:
for i in range(2):
   #print('conv%d' % (i))
   #print('conv%d_dynamic' % i)
   #print('W_%d_dynamic' % i)
   #print('W1_%d_dynamic' % i)
   print('W2_%d_dynamic' % i)

W2_0_dynamic
W2_1_dynamic


In [13]:
#测试5：flow_dim
num_out_channel=256
GROUP_SIZE=3
flow_dim = num_out_channel // 2 // GROUP_SIZE * GROUP_SIZE + GROUP_SIZE
print(flow_dim)

129


In [ ]:
#测试6：测试tf.concat
import tensorflow as tf
batch_size=1
npoint=9
nsample=4

a=4
b=5
out_channel=a
new_points=tf.zeros([batch_size,npoint,nsample,out_channel])
new_points_re=tf.reshape(new_points,[batch_size, -1, a])
grouped_time=tf.zeros([batch_size,npoint,nsample,b])

ccat1=tf.concat([new_points, grouped_time], -1)


with tf.Session() as sess:
    print("new_points.shape:",sess.run(new_points).shape)
    print("new_points_re.shape:",sess.run(new_points_re).shape)
    print("grouped_time.shape:",sess.run(grouped_time).shape)
    print("tf.concat结果的shape:",sess.run(ccat1).shape)


In [ ]:
export CUDA_HOME=$CUDA_HOME:/usr/local/cuda-11.0
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-11.0/lib64
export PATH=$PATH:$CUDA_HOME/bin
source ~/.bashrc

In [ ]:
#测试7：normals_feat
import tensorflow as tf
batch_size=1
npoint=2
flow_dim=6
GROUP_SIZE=3
normals_feat=tf.zeros([batch_size,npoint,flow_dim // GROUP_SIZE,GROUP_SIZE]) #shape(1,2,2,3)
b=tf.ones(shape=[batch_size, npoint, flow_dim // GROUP_SIZE, 1], dtype=normals_feat.dtype)#(1,2,2,1)
c=tf.concat([normals_feat, -tf.ones(shape=[batch_size, npoint, flow_dim // GROUP_SIZE, 1], dtype=normals_feat.dtype)], -1)
normals_feat=c
normals_feat_final=tf.reshape(normals_feat, [batch_size, npoint, -1])
with tf.Session() as sess:
    print("normals_feat:【shape{}】\n{}".format(sess.run(normals_feat).shape,sess.run(normals_feat)))
    print("b:【shape{}】\n{}".format(sess.run(b).shape,sess.run(b)))
    #print("b.shape:",sess.run(b).shape)
    print("c:【shape{}】\n{}".format(sess.run(c).shape,sess.run(c)))
    #print("c.shape:",sess.run(c).shape)
    print("normals_feat_final:【shape{}】\n{}".format(sess.run(normals_feat_final).shape,sess.run(normals_feat_final)))


In [9]:
#测试8：tf.matmul
import tensorflow as tf

a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2])
c = tf.matmul(a, b)
with tf.Session() as sess:
    print(sess.run([a,b,c]))


[array([[1, 2, 3],
       [4, 5, 6]]), array([[ 7,  8],
       [ 9, 10],
       [11, 12]]), array([[ 58,  64],
       [139, 154]])]


In [1]:
#测试8：tf.matmul 5维矩阵 相乘维度
#测试9：get_st_surface
import tensorflow as tf
batch_size=1
npoint=2
Group_num=11
nsample=64
GROUP_SIZE=3

fea=tf.ones(shape=[batch_size, npoint, Group_num,nsample, GROUP_SIZE+1], dtype=b.dtype) #shape=[1, 2, 11, 64, 4]
AtA=tf.matmul(fea, fea, transpose_a=True)
new_points_=fea
#feature matirx  shape=[1, 2, 11, 64, 4]
duplicate_grouped_time=tf.ones(shape=[batch_size, npoint, Group_num,nsample, 1], dtype=b.dtype)
#time  matirx    shape=[1, 2, 11, 64, 1]
n_new_points_ = tf.matmul(new_points_, duplicate_grouped_time, transpose_a=True)
with tf.Session() as sess:
    sess.run([fea,AtA,new_points_,duplicate_grouped_time,n_new_points_])
    print("duplicate_grouped_time.shape:",duplicate_grouped_time.shape)
    print("fea.shape:",fea.shape)
    print("AtA.shape:",AtA.shape)
    print("n_new_points_.shape:",n_new_points_.shape)


NameError: name 'b' is not defined

In [13]:
#测试tf.unstack
import tensorflow as tf
import numpy as np

sess = tf.Session()
sess.run(tf.global_variables_initializer())

ust_data = np.arange(1, 61).reshape([1, 3, 4, 5])
print('ust_data: \n', ust_data)
print('ust_data.shape: \n', ust_data.shape)

print('======================================')

ust_3 = tf.unstack(ust_data, axis=3)
ust_3 = sess.run(ust_3)
print('ust_3: ', ust_3)
print('ust_3.length: ', len(ust_3))
print('ust_3.shape: ', len(ust_3), ' * ', ust_3[0].shape)       #  5 * (1, 3, 4) ==> (5, 2, 3, 4)


# 理解：
    # tf.unstack 其实是将axis维度直接放到最前面
    # 也和 tf.transpose 类似
#      总结：
#     tf.unstack() 中 stacks = (维1，维2， 维3， 维4 ）
#     当axis=0时， 就相当于tf.transpose(stacks, [0， 1， 2， 3])
#     当axis=1时， 就相当于tf.transpose(stacks, [1， 0， 2， 3])
#     当axis=2时， 就相当于tf.transpose(stacks, [2， 0， 1， 3])
#     当axis=3时， 就相当于tf.transpose(stacks, [3， 0， 1， 2])

ust_data: 
 [[[[ 1  2  3  4  5]
   [ 6  7  8  9 10]
   [11 12 13 14 15]
   [16 17 18 19 20]]

  [[21 22 23 24 25]
   [26 27 28 29 30]
   [31 32 33 34 35]
   [36 37 38 39 40]]

  [[41 42 43 44 45]
   [46 47 48 49 50]
   [51 52 53 54 55]
   [56 57 58 59 60]]]]
ust_data.shape: 
 (1, 3, 4, 5)
ust_3:  [array([[[ 1,  6, 11, 16],
        [21, 26, 31, 36],
        [41, 46, 51, 56]]]), array([[[ 2,  7, 12, 17],
        [22, 27, 32, 37],
        [42, 47, 52, 57]]]), array([[[ 3,  8, 13, 18],
        [23, 28, 33, 38],
        [43, 48, 53, 58]]]), array([[[ 4,  9, 14, 19],
        [24, 29, 34, 39],
        [44, 49, 54, 59]]]), array([[[ 5, 10, 15, 20],
        [25, 30, 35, 40],
        [45, 50, 55, 60]]])]
ust_3.length:  5
ust_3.shape:  5  *  (1, 3, 4)


In [4]:
#测试W=last_W[:,:,0,:]是否会降维
import tensorflow as tf
last_W=tf.ones(shape=[2, 3, 4,5], dtype=b.dtype)
W=last_W[:,:,0,:]

with tf.Session() as sess:
    sess.run([last_W,W])
    print("W=last_W[:,:,0,:]的形状：",W.shape)

W=last_W[:,:,0,:]的形状： (2, 3, 5)


In [5]:
#测试idx:方法sample_and_group_all的返回参数
import tensorflow as tf
import numpy as np
nsample=5
batch_size=2

idx = tf.constant(np.tile(np.array(range(nsample)).reshape((1,1,nsample)), (batch_size,1,1)))
with tf.Session() as sess:
    print("nsample={},batch_size={}".format(nsample,batch_size))
    idx_t=sess.run(idx)
    print("idx：\n",idx_t)
    print("idx的形状：",idx.shape)


nsample=5,batch_size=2
idx：
 [[[0 1 2 3 4]]

 [[0 1 2 3 4]]]
idx的形状： (2, 1, 5)
5


In [8]:
flow1_list=[[1,2,3],[2,3]]
len(flow1_list)

2

In [10]:
import tensorflow as tf
flow1_time=tf.ones(shape=[2, 3, 4,5], dtype=b.dtype)
#tf.concat([last_flow1, flow1_time])

NameError: name 'last_flow1' is not defined

In [11]:
mlp=(11*4, )
mlp[-1]

44

In [3]:
#测试3维矩阵reshape
import numpy as np

a=np.random.randint(0,10,size=[1,3,3*2])
print("a:{}\n{}".format(a.shape,a))
b=a.reshape(1,-1,2)
print("b:{}\n{}".format(b.shape,b))


a:(1, 3, 6)
[[[7 5 6 8 4 3]
  [3 0 5 1 9 7]
  [7 4 2 2 1 4]]]
b:(1, 9, 2)
[[[7 5]
  [6 8]
  [4 3]
  [3 0]
  [5 1]
  [9 7]
  [7 4]
  [2 2]
  [1 4]]]


In [ ]:
pathroot=os.path.dirname(__file__)#去掉文件名后的绝对路径
path2data=os.path.join(pathroot,"..","data","HPLFlowNet")

In [ ]:
sudo apt-get --purge remove "cuda*"
sudo apt-get --purge remove "*nvidia*"

sudo apt-get install -y nvidia-docker2
sudo systemctl restart docker
vim ~/.bashrc
source ~/.bashrc
sudo apt-get install build-essential
sudo apt-get install aptitude
sudo aptitude install build-essential
sudo tar -xvzf cudnn-11.2-linux-x64-v8.1.1.33.tgz
sudo cp cuda/include/cudnn* /usr/local/cuda-11.2/include
sudo cp cuda/lib64/libcudnn* /usr/local/cuda-11.2/lib64
sudo chmod a+r /usr/local/cuda-11.2/include/cudnn*
sudo chmod a+r /usr/local/cuda-11.2/lib64/libcudnn*
cat /usr/local/cuda-11.2/include/cudnn.h
python
import torch
print(torch.__version__)
torch.cuda.get_arch_list()
torch.cuda.get_device_capability()

In [ ]:
def changed_pointnet_sa_module(points, npoint, mlp, mlp2,  is_training, bn_decay, scope, bn=True, pooling='max', knn=False, use_xyz=True, use_nchw=False, freeze_bn=False,  trainable=True):
    ''' changed PointNet Set Abstraction (SA) Module
        Input:

            points: (batch_size, ndataset, channel) TF tensor
            npoint: int32 -- #points sampled in farthest point sampling

            mlp: list of int32 -- output size for MLP on each point
            mlp2: list of int32 -- output size for MLP on each region

            use_xyz: bool, if True concat XYZ with local point features, otherwise just use point features
            use_nchw: bool, if True, use NCHW data format for conv2d, which is usually faster than NHWC format
        Return:
            new_points: (batch_size, npoint, mlp[-1] or mlp2[-1]) TF tensor
    '''
    data_format = 'NCHW' if use_nchw else 'NHWC' #NHWC
    with tf.variable_scope(scope) as sc:
        new_points = tf.expand_dims(points, 1) # (batch_size, 1, 32*16, 3)#points:(batch_size,32*16, 3)

        # Point Feature Embedding
        if use_nchw: new_points = tf.transpose(new_points, [0,3,1,2])
        for i, num_out_channel in enumerate(mlp):
            new_points = tf_util.conv2d(new_points, num_out_channel, [1,1],
                                        padding='VALID', stride=[1,1],
                                        bn=bn, is_training=is_training, trainable=trainable,
                                        scope='conv%d'%(i), bn_decay=bn_decay,
                                        data_format=data_format, freeze_bn=freeze_bn)
            # data_format = 'NHWC' ，因为kernel_size[1,1], padding='VALID', stride=[1,1]
            # 所以，shape中改变的只有C。
            #new_points:shape(batch_size,1,32*16,mlp[-1])
        if use_nchw: new_points = tf.transpose(new_points, [0,2,3,1])

        # Pooling in Local Regions
        if pooling=='max':
            new_points = tf.reduce_max(new_points, axis=[2], keepdims=True, name='maxpool')
            #shape(batch_size,1, 1,mlp[-1])
            #比如mlp=[512, 1024],group_all=True,shape(batch_size,1, 1,1024)
        elif pooling=='avg':
            new_points = tf.reduce_mean(new_points, axis=[2], keepdims=True, name='avgpool')
        elif pooling=='weighted_avg':
            with tf.variable_scope('weighted_avg'):
                dists = tf.norm(grouped_xyz,axis=-1,ord=2,keepdims=True)
                exp_dists = tf.exp(-dists * 5)
                weights = exp_dists/tf.reduce_sum(exp_dists,axis=2,keepdims=True) # (batch_size, npoint, nsample, 1)
                new_points *= weights # (batch_size, npoint, nsample, mlp[-1])
                new_points = tf.reduce_sum(new_points, axis=2, keepdims=True)
        elif pooling=='max_and_avg':
            max_points = tf.reduce_max(new_points, axis=[2], keepdims=True, name='maxpool')
            avg_points = tf.reduce_mean(new_points, axis=[2], keepdims=True, name='avgpool')
            new_points = tf.concat([avg_points, max_points], axis=-1)

        # [Optional] Further Processing
        if mlp2 is not None:
            if use_nchw: new_points = tf.transpose(new_points, [0,3,1,2])
            for i, num_out_channel in enumerate(mlp2):
                new_points = tf_util.conv2d(new_points, num_out_channel, [1,1],
                                            padding='VALID', stride=[1,1],
                                            bn=bn, is_training=is_training, trainable=trainable,
                                            scope='conv_post_%d'%(i), bn_decay=bn_decay,
                                            data_format=data_format, freeze_bn=freeze_bn)
            if use_nchw: new_points = tf.transpose(new_points, [0,2,3,1])

        new_points = tf.squeeze(new_points, [2]) # (batch_size, npoints, mlp2[-1])
        ##比如mlp=[512, 1024],group_all=True,shape(batch_size,1, 1024)

        return  new_points



In [ ]:
r=1
print("r="+r)